## Convolution NNet | Сверточная сеть

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn import  metrics

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adadelta
import tensorflow.keras.utils as np_utils

In [3]:
train = pd.read_csv("train.csv")
Y = train['label']
X = train.drop(['label'], axis=1)

In [4]:
# Разделяем на обучающую выборку и выборку валидации
x_train, x_val, y_train, y_val = train_test_split(X.values, Y.values, test_size=0.10, random_state=42)

In [5]:
batch_size = 128
num_classes = 10
epochs = 5 

# Размерность картинки
img_rows, img_cols = 28, 28

In [6]:
# Преобразование обучающей выборки
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_train /= 255

# Преобразование выборки валидации
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
x_val = x_val.astype('float32')
x_val /= 255

In [7]:
input_shape = (img_rows, img_cols, 1)

# Преобразование отклика в 10 бинарных перменных 
y_train = np_utils.to_categorical(y_train, num_classes)
y_val   = np_utils.to_categorical(y_val, num_classes)

In [8]:
model = Sequential()

# Первый сверточный слой
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# Второй сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu'))

# Слой Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))

# Слой dropout
model.add(Dropout(0.25))

# Растягиваем в вектор
model.add(Flatten())

# Первый слой анализа 
model.add(Dense(128, activation='relu'))

# Слой dropout
model.add(Dropout(0.5))

# Второй слой анализа 
model.add(Dense(num_classes, activation='softmax'))

# Обучение
model.compile(loss='categorical_crossentropy',  
              optimizer=Adadelta(), 
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [9]:
# Warning! Будет долго считаться
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_val, y_val))
accuracy = model.evaluate(x_val, y_val, verbose=0)
print('Test accuracy:', accuracy[1])

Epoch 1/5
296/296 [==============================] - 41s 139ms/step - loss: 2.2849 - accuracy: 0.1335 - val_loss: 2.2602 - val_accuracy: 0.2324
Epoch 2/5
296/296 [==============================] - 44s 148ms/step - loss: 2.2524 - accuracy: 0.1992 - val_loss: 2.2234 - val_accuracy: 0.3095
Epoch 3/5
296/296 [==============================] - 47s 159ms/step - loss: 2.2165 - accuracy: 0.2547 - val_loss: 2.1815 - val_accuracy: 0.3950
Epoch 4/5
296/296 [==============================] - 39s 132ms/step - loss: 2.1751 - accuracy: 0.3082 - val_loss: 2.1318 - val_accuracy: 0.4831
Epoch 5/5
296/296 [==============================] - 39s 131ms/step - loss: 2.1250 - accuracy: 0.3620 - val_loss: 2.0717 - val_accuracy: 0.5533
Test accuracy: 0.5533333420753479
